<a href="https://colab.research.google.com/github/Siyu0011/Burst_dectection/blob/main/Copy_of_Siyu_burst_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install hmmlearn
!pip install mne

from hmmlearn import hmm

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
#from hmmlearn import hmm
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
from scipy import signal
from numpy.random import default_rng
from hmmlearn import hmm

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving s001.csv to s001.csv


In [ ]:
#segmentation
import pandas as pd 
df = pd.read_csv ('s001.csv',header=None)
#select all-0 line and delete
df=df.loc[~(df==0).all(axis=1)] 

lo = int((df.shape[1])/530)                 ## 530 means the time point of each epoch
from typing import List
ro = int(len(df) * lo + 1)

res1 = [[0] * 3 for i in range(ro)]
#res1= []
res1[0].append('rate')
res1[0].append('ampl')
res1[0].append('duration')
for tt in range (1,lo+1) :
  tt1 = tt*530-530                          ## 530 means the time point of each epoch
  tt2 = tt1 + 530
  df1 = df.iloc[:,tt1:tt2]
  interval = len(df1.columns)/250           ## 250 is the sample rate

  for tr in range (0,len(df1)):
  
    z = []
    q = []
    w = []
    b_state = []
    tr1 = len(df1)*(tt-1) + tr + 1

  ## Extracting beta wave from raw MEG signal
    sos = signal.butter(5, [15, 29], 'bandpass', fs=250, output='sos')     ## 250 is the sample rate
    filtered = signal.sosfilt(sos,df1.iloc[tr])
    analytic_signal = signal.hilbert(filtered)
    amplitude_envelope = np.abs(analytic_signal)
    amplitude_envelope = signal.detrend(amplitude_envelope, type = 'linear')
    threshold = np.percentile(amplitude_envelope,75)
    z = amplitude_envelope>threshold

  ## HMM implemetation 

    remodel = hmm.GMMHMM(n_components=2,covariance_type="full", n_iter=100000)
    remodel.fit(amplitude_envelope.reshape(-1, 1))
    proba = remodel.predict_proba(amplitude_envelope.reshape(-1, 1))

  ## setting state_prob == 0.66

    q =proba[:, 0]
    q[q > 0.66] = 1
    q[q < 0.66] = 0

    w=proba[:, 1]
    w[w > 0.66] = 1
    w[w < 0.66] = 0

  
    if np.mean(w!=z) > np.mean(q!=z):
      b_state = q
    else:
      b_state = w
  
    j_1=0
  
  ## Computing burst charcateristics (rate, ampl, duration) 

    from itertools import groupby
    grouped_l= []
    grouped_l = [k for k,g in groupby(b_state)]

    grouped_l1= []
    grouped_l3= []
    grouped_l4= []
    for k,g in groupby(b_state):
      j_1 = j_1 +sum(1 for i in g)
      grouped_l1.append (j_1)
    o=0
    bamp=0
    bduration_1 = 0
    bamp_f_1 =0
    if grouped_l[0]==1:
      o=0
      bamp =0
      o= sum(b_state[0:grouped_l1[0]])
    
      bamp = bamp+ max(amplitude_envelope[0:grouped_l1[0]])
      if o > 25:                                                      ## 25 means time points in 100ms (sample rate/10)

        grouped_l3.append(o)
        grouped_l4.append(bamp)
      for i in range(1,len(grouped_l1)-1,2):
        o = 0
        bamp = 0
        o = o+ sum(b_state[grouped_l1[i]:grouped_l1[i+1]])
        bamp = bamp+ max(amplitude_envelope[grouped_l1[i]:grouped_l1[i+1]])
    #o = o+ sum(q[(i-1):grouped_L1[i]])
      
  ## setting minimum burst duration threshold, 12 means 12/sampling freq = 100 ms;     
      
        if o > 25:
          grouped_l3.append(o)
          grouped_l4.append(bamp)
        o = 0

    else:
      for i in range(0,len(grouped_l1)-1,2):
        o = 0
        bamp = 0
        o = o+ sum(b_state[grouped_l1[i]:grouped_l1[i+1]])
        bamp = bamp+ max(amplitude_envelope[grouped_l1[i]:grouped_l1[i+1]])
      #o = o+ sum(q[(i-1):grouped_L1[i]])
        if o > 25:
          grouped_l3.append(o)
          grouped_l4.append(bamp)
        o=0
    res1[tr1].append(len(grouped_l3)/interval)
    if sum(grouped_l3) ==0:
      bamp_f_1 = 0
      bduration_1 =0
    else:
  
      bamp_f_1 =  grouped_l4
      bduration_1 = sum(grouped_l3)/len(grouped_l3)
    res1[tr1].append(np.mean(bamp_f_1))
  
    res1[tr1].append(bduration_1*4)

import numpy as np
np.savetxt("s001_result.csv", res1, delimiter=",", fmt='%s')
files.download('s001_result.csv')  ## each row of csv file corresponds to burst characteristics of each trial of the selected interval 

In [ ]:
reset                  ## it's better to clear all the variables after each subject

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [ ]:
import pandas as pd 
df = pd.read_csv ('s001.csv',header=None)
#select all-0 line and delete
df=df.loc[~(df==0).all(axis=1)] 
lo = (df.shape[1])/530


In [ ]:
# whole session
import pandas as pd 
df = pd.read_csv ('s002.csv',header=None)
#select all-0 line and delete
df=df.loc[~(df==0).all(axis=1)] 

from typing import List
ro = int(len(df) + 1)
res1 = [[0] * 3 for i in range(ro)]
res2 = np.empty((4,df.shape[1]))
#res1= []
res1[0].append('rate')
res1[0].append('ampl')
res1[0].append('duration')

interval = len(df.columns)/250
for tr in range (0,len(df)):
  
  z = []
  q = []
  w = []
  b_state = []


  ## Extracting beta wave from raw MEG signal
  sos = signal.butter(5, [15, 29], 'bandpass', fs=250, output='sos')
  filtered = signal.sosfilt(sos,df.iloc[tr])
  analytic_signal = signal.hilbert(filtered)
  amplitude_envelope = np.abs(analytic_signal)
  amplitude_envelope = signal.detrend(amplitude_envelope, type = 'linear')
  threshold = np.percentile(amplitude_envelope,75)
  z = amplitude_envelope>threshold

  ## HMM implemetation 

  remodel = hmm.GMMHMM(n_components=2,covariance_type="full", n_iter=100)
  remodel.fit(amplitude_envelope.reshape(-1, 1))
  proba = remodel.predict_proba(amplitude_envelope.reshape(-1, 1))

  ## setting state_prob == 0.66

  q =proba[:, 0]
  q[q > 0.66] = 1
  q[q < 0.66] = 0

  w=proba[:, 1]
  w[w > 0.66] = 1
  w[w < 0.66] = 0

  
  if np.mean(w!=z) > np.mean(q!=z):
    b_state = q
  else:
    b_state = w
  
  j_1=0
  
  ## Computing burst charcateristics (rate, ampl, duration) 

  from itertools import groupby
  grouped_l= []
  grouped_l = [k for k,g in groupby(b_state)]

  grouped_l1= []
  grouped_l3= []
  grouped_l4= []
  for k,g in groupby(b_state):
    j_1 = j_1 +sum(1 for i in g)
    grouped_l1.append (j_1)
  o=0
  bamp=0
  bduration_1 = 0
  bamp_f_1 =0
  if grouped_l[0]==1:
    o=0
    bamp =0
    o= sum(b_state[0:grouped_l1[0]])
    
    bamp = bamp+ max(amplitude_envelope[0:grouped_l1[0]])
    if o > 25:

      grouped_l3.append(o)
      grouped_l4.append(bamp)
    for i in range(1,len(grouped_l1)-1,2):
      o = 0
      bamp = 0
      o = o+ sum(b_state[grouped_l1[i]:grouped_l1[i+1]])
      bamp = bamp+ max(amplitude_envelope[grouped_l1[i]:grouped_l1[i+1]])
    #o = o+ sum(q[(i-1):grouped_L1[i]])
      
  ## setting minimum burst duration threshold, 12 means 12/sampling freq = 100 ms;     
      
      if o > 25:
        grouped_l3.append(o)
        grouped_l4.append(bamp)
      o = 0

  else:
    for i in range(0,len(grouped_l1)-1,2):
      o = 0
      bamp = 0
      o = o+ sum(b_state[grouped_l1[i]:grouped_l1[i+1]])
      bamp = bamp+ max(amplitude_envelope[grouped_l1[i]:grouped_l1[i+1]])
      #o = o+ sum(q[(i-1):grouped_L1[i]])
      if o > 25:
        grouped_l3.append(o)
        grouped_l4.append(bamp)
      o=0
  res1[tr+1].append(len(grouped_l3)/interval)
  if sum(grouped_l3) ==0:
    bamp_f_1 = 0
    bduration_1 =0
  else:
  
    bamp_f_1 =  grouped_l4
    bduration_1 = sum(grouped_l3)/len(grouped_l3)
  res1[tr+1].append(np.mean(bamp_f_1))
  res1[tr+1].append(bduration_1*4)
  res2[tr,:] = b_state                       ## res2 refers to all the labels in the data set (continuous)
import numpy as np
np.savetxt("s002_result.csv", res1, delimiter=",", fmt='%s')
files.download('s002_result.csv')  ## each row of csv file corresponds to burst characteristics of each trial of the selected interval 

In [ ]:
reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
